# documents

In [ ]:
open file_system_operators
open sm'_operators
open rust_operators

()



In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    rust.types ()
    am'.types ()
    sm'.types ()
    file_system.types ()
    runtime.types ()
    async.types ()

()



## args

In [ ]:
inl get_args () =
    {
        source_dir = "source-dir"
        dist_dir = "dist-dir"
        cache_dir = "cache-dir"
        hangul_spec = "hangul-spec"
    }

()



## get_command

In [ ]:
let get_command () =
    ##"command"
    |> runtime.new_command
    |> runtime.command_arg (
        runtime.new_arg ##(get_args () .source_dir)
        |> runtime.arg_short 's'
        |> runtime.arg_long ##(get_args () .source_dir)
        |> runtime.arg_required true
    )
    |> runtime.command_arg (
        runtime.new_arg ##(get_args () .dist_dir)
        |> runtime.arg_short 'd'
        |> runtime.arg_long ##(get_args () .dist_dir)
        |> runtime.arg_required true
    )
    |> runtime.command_arg (
        runtime.new_arg ##(get_args () .cache_dir)
        |> runtime.arg_short 'c'
        |> runtime.arg_long ##(get_args () .cache_dir)
        |> runtime.arg_required true
    )
    |> runtime.command_arg (
        runtime.new_arg ##(get_args () .hangul_spec)
        |> runtime.arg_short 's'
        |> runtime.arg_long ##(get_args () .hangul_spec)
        |> runtime.arg_required true
    )

()



## crowbook

In [ ]:
let crowbook { ext dist_dir dist_path output_path } =
    inl command = $'$"crowbook --single \\\"{!dist_path}\\\" --output \\\"{!output_path}\\\" --to {!ext} --set rendering.num_depth 6 html.css.add \\\\\\"\'\'\' body {{ color: #e8e6e3; background-color: #202324; }} a {{ color: #989693; }} \'\'\'\\\\\\""'

    inl exit_code, result =
        runtime.execution_options fun x => { x with
            command
            working_directory = dist_dir |> Some |> optionm'.box
        }
        |> runtime.execute_with_options

    if result |> sm'.contains "ERROR" then
        trace Info
            fun () => $'"documents.crowbook / result contains ERROR"'
            fun () => $'$"exit_code: %A{!exit_code} / result: {!result} / output_path: {!output_path} / {!_locals ()}"'
        (exit_code, result) |> Error
    else (exit_code, result) |> Ok

()



## hangul

In [ ]:
let hangul repository_root hangul_spec { ext dist_dir dist_path output_path } =
    inl lines =
        dist_path
        |> file_system.read_all_text
        |> sm'.split "\n"
        |> fun x => a x : _ i32 _
        |> am.map sm'.trim
    inl text =
        lines
        |> am.filter ((<>.) "")
        |> seq.of_array'
        |> sm'.concat "\n"
        |> fun x => $'$"{!x}\n\n"'

    inl exit_code, result =
        runtime.execution_options fun x => { x with
            command =
                inl hangulize_path =
                    inl _exe = runtime.get_executable_suffix ()
                    repository_root </> $'$"../vault/deps/hangulize/cmd/hangulize/hangulize{!_exe}"'
                $'$"{!hangulize_path} {!hangul_spec}"'
            stdin =
                fun stdin =>
                    inl stdin =
                        stdin
                        |> threading.arc_mutex_lock
                        |> resultm.unwrap'
                    text |> runtime.stdin_write_all stdin
                |> Some |> optionm'.box
        }
        |> runtime.execute_with_options

    inl result : string =
        inl result =
            result
            |> sm'.split "\n"
            |> fun x => a x : _ i32 _
        inl result_len = result |> am'.length

        (("", (0i32, 0i32)), lines)
        ||> am.fold fun (acc, (i, n)) x =>
            if x = ""
            then $'$"{!acc}\n"', (i + 1, n + 1)
            else
                inl acc =
                    inl i = i - n
                    if i >= result_len
                    then acc
                    else
                        inl line = result |> am'.index i
                        if i = result_len - 1
                        then $'$"{!acc}{!line}"'
                        else $'$"{!acc}{!line}\n"'
                acc, (i + 1, n)
        |> fst

    result |> file_system.write_all_text output_path

    trace Info
        fun () => $'"documents.hangul"'
        fun () =>
            inl result_len : i32 = result |> sm'.length
            $'$"exit_code: %A{!exit_code} / result_len: {!result_len} / output_path: {!output_path} / {!_locals ()}"'

    (exit_code, result) |> Ok
    |> fun x => x : result (i32 * string) (i32 * string)

()



## fix_paths

In [ ]:
let fix_paths { cache_dir dist_path ext } =
    inl file_name = dist_path |> file_system.get_file_name
    inl cache_path = cache_dir </> file_name
    inl dist_path_no_ext =
        dist_path
        |> sm'.slice 0 ((dist_path |> sm'.last_index_of ".") - 1)
    inl cache_path_no_ext =
        cache_path
        |> sm'.slice 0 ((cache_path |> sm'.last_index_of ".") - 1)

    inl is_md = ext |> sm'.ends_with ".md"
    inl output_path : string =
        if is_md |> not
        then $'$"{!dist_path}.{!ext}"'
        else $'$"{!dist_path_no_ext}.{!ext}"'
    inl output_cache_path : string =
        if is_md |> not
        then $'$"{!cache_path}.{!ext}"'
        else $'$"{!cache_path_no_ext}.{!ext}"'
    { output_path output_cache_path }

()



## files_fn

In [ ]:
inl files_fn { dist_dir cache_dir } fn dist_path ext =
    inl { output_path output_cache_path } = fix_paths { cache_dir dist_path ext }
    inl equal =
        if file_system.file_exists output_path
            && file_system.file_exists output_cache_path then
            inl output_hash = output_path |> crypto.get_file_hash |> resultm.get
            inl output_cache_hash = output_cache_path |> crypto.get_file_hash |> resultm.get
            output_hash = output_cache_hash
        else false

    if not equal
    then
        match fn { ext dist_dir dist_path output_path } with
        | Ok (exit_code, result) when exit_code <>. 0 =>
            trace Info
                fun () => $'"documents.files_fn / error"'
                fun () => $'$"exit_code: %A{!exit_code} / result: {!result} / {!_locals ()}"'
            new_pair output_path result |> Error |> resultm.box |> Some
        | Error (exit_code, result) =>
            new_pair output_path result |> Error |> resultm.box |> Some
        | _ =>
            if output_path |> file_system.file_exists
            then output_path |> file_system.file_copy output_cache_path
            else failwith $'$"documents.files_fn / {!output_path} should exist"'
            output_path |> Ok |> resultm.box |> Some
    else None

()



## run

In [ ]:
let run { source_dir dist_dir cache_dir hangul_spec }
    : async.future_pin (resultm.result' (am'.vec (resultm.result' (pair string (am'.vec (optionm'.option' (resultm.result' string (pair string string))))) sm'.std_string)) sm'.std_string) =
    
    inl repository_root = file_system.get_repository_root ()

    inl source_dir = source_dir |> file_system.get_full_path
    inl dist_dir = dist_dir |> file_system.get_full_path
    inl cache_dir = cache_dir |> file_system.get_full_path
    
    trace Debug
        fun () => $'"documents.run"'
        fun () => $'$"source_dir: {!source_dir} / dist_dir: {!dist_dir} / cache_dir: {!cache_dir} / hangul_spec: %A{!hangul_spec} / {!_locals ()}"'
    
    fun () =>
        inl files =
            dist_dir
            |> file_system.new_walk_dir
            |> file_system.walk_dir_filter fun entry => async.future_init_send (2, 1) 1 fun () =>
                entry
                |> file_system.dir_entry_file_type
                |> async.await_send
                |> resultm.map_error' sm'.format'
                |> resultm.unbox
                |> function
                    | Ok file_type when file_type |> file_system.file_type_is_dir =>
                        file_system.Ignore
                    | _ =>
                        inl path =
                            entry
                            |> file_system.dir_entry_path
                            |> file_system.path_buf_display
                            |> sm'.format'
                            |> sm'.from_std_string
                        if (path |> sm'.ends_with ".md" |> not) || (path |> sm'.ends_with ".hangul.md")
                        then file_system.Ignore
                        else file_system.Continue
            |> file_system.stream_filter_map fun entry =>
                match entry |> resultm.map_error' sm'.format' |> resultm.unbox with
                | Ok entry =>
                    entry
                    |> file_system.dir_entry_path
                    |> file_system.path_buf_display
                    |> sm'.format'
                    |> sm'.from_std_string
                    |> Some
                | Error error =>
                    trace Critical
                        fun () => $'"documents.run / stream_filter_map"'
                        fun () => $'$"error: {!error} / {!_locals ()}"'
                    None
                |> optionm'.box

        trace Debug
            fun () => $'"documents.run"'
            fun () =>
                inl files_len = files |> am'.vec_len
                $'$"files_len: {!files_len} / {!_locals ()}"'

        inl result =
            files
            |> async.into_par_iter
            |> async.par_map fun file =>
                inl file = file |> file_system.get_full_path

                inl relative_path =
                    file
                    |> sm'.to_std_string
                    |> file_system.new_path_buf
                    |> file_system.path_buf_display
                    |> sm'.format'
                    |> sm'.from_std_string
                    |> sm'.replace dist_dir ""
                    |> sm'.replace "\\" "/"
                    |> fun s => $'$".{!s}"'

                inl file = file |> file_system.normalize_path

                inl real_path = source_dir </> relative_path |> file_system.normalize_path

                inl origin_hash_exit_code, origin_hash =
                    runtime.execution_options fun x => { x with
                        command = $'$"git ls-tree --format=\'%%(objectname)\' origin/gh-pages \\"{!real_path}\\""'
                        working_directory = source_dir |> Some |> optionm'.box
                    }
                    |> runtime.execute_with_options
                
                inl dist_path = dist_dir </> relative_path |> file_system.normalize_path

                inl local_git_hash_exit_code, local_git_hash =
                    runtime.execution_options fun x => { x with
                        command = $'$"git hash-object \\"{!dist_path}\\""'
                        working_directory = dist_dir |> Some |> optionm'.box
                    }
                    |> runtime.execute_with_options

                inl cache_path = cache_dir </> relative_path |> file_system.normalize_path

                inl files =
                    inl files_fn = files_fn { dist_dir cache_dir }
                    inl files = [
                        "hangul.md", dist_path, hangul repository_root hangul_spec |> files_fn
                    ]

                    inl { output_path } = fix_paths { dist_path cache_dir ext = "hangul.md" }

                    inl files' = [
                        "html", dist_path, crowbook |> files_fn
                        "pdf", dist_path, crowbook |> files_fn
                        "epub", dist_path, crowbook |> files_fn
                        "html", output_path, crowbook |> files_fn
                        "pdf", output_path, crowbook |> files_fn
                        "epub", output_path, crowbook |> files_fn
                    ]

                    [ files; files' ]

                inl files' =
                    if origin_hash |> sm'.contains local_git_hash |> not then
                        inl hash1 = dist_path |> crypto.get_file_hash |> resultm.get
                        inl hash2 =
                            if cache_path |> file_system.file_exists
                            then cache_path |> crypto.get_file_hash |> resultm.ok
                            else None

                        if hash2 = None || hash1 <>. (hash2 |> optionm.value) then
                            trace Info
                                fun () => $'"documents.run / par_map / origin_hash |> sm\'.contains local_git_hash |> not / hash2 = None || hash1 <>. (hash2 |> optionm.value)"'
                                fun () => $'$"file: {!file} / real_path: {!real_path} / relative_path: {!relative_path} / origin_hash_exit_code: {!origin_hash_exit_code} / origin_hash: {!origin_hash} / local_git_hash_exit_code: {!local_git_hash_exit_code} / local_git_hash: {!local_git_hash} / hash1: {!hash1} / hash2: %A{!hash2} / dist_path: {!dist_path} / cache_path: {!cache_path} / {!_locals ()}"'
                            dist_path |> file_system.file_copy cache_path
                            files
                        else []
                    else []
                    
                inl files' =
                    if files' <> []
                    then files'
                    else
                        files
                        |> listm.map fun files =>
                            files
                            |> listm'.filter fun ext, path, fn =>
                                inl { output_path output_cache_path } =
                                    fix_paths { cache_dir dist_path = path; ext }
                                if output_path |> file_system.file_exists |> not then
                                    if file_system.file_exists output_cache_path then
                                        trace Info
                                            fun () => $'"documents.run / par_map / files\' = [] / listm.iter"'
                                            fun () => $'$"output_path: %A{!output_path} / output_cache_path: {!output_cache_path} / {!_locals ()}"'
                                        output_cache_path |> file_system.file_copy output_path
                                        false
                                    else true
                                else true

                inl files =
                    files'
                    |> listm'.box
                    |> listm'.to_array'
                    |> fun (a x : _ i32 _) => x
                    |> am'.to_vec
                    |> am'.vec_collect fun files =>
                        files
                        |> listm'.box
                        |> listm'.to_array'
                        |> fun (a x : _ i32 _) => x
                        |> am'.to_vec
                        |> async.into_par_iter
                        |> async.par_map fun ext, path, fn =>
                            fn path ext |> optionm'.box
                        |> async.par_collect

                (new_pair file files |> Ok |> resultm.box) : _ _ sm'.std_string
            |> async.par_collect

        trace Debug
            fun () => $'"documents.run"'
            fun () =>
                inl result_len = result |> am'.vec_len
                $'$"result_len: {!result_len} / {!_locals ()}"'

        result |> Ok |> resultm.box
    |> async.future_init (4, 3) 1

()



In [ ]:
//// test
//// print_code=false
///! rust

types ()
inl repository_root = file_system.get_repository_root ()
inl source_dir = repository_root </> "../vault/target/documents"
inl dist_dir = source_dir </> "dist"
inl cache_dir = source_dir </> "cache"
inl file_name_no_ext = "test"
inl file_name = join $'$"{!file_name_no_ext}.md"'

source_dir |> file_system.directory_delete true
dist_dir |> file_system.directory_delete true
cache_dir |> file_system.directory_delete true

source_dir |> file_system.create_directory' |> ignore
dist_dir |> file_system.create_directory' |> ignore
cache_dir |> file_system.create_directory' |> ignore

inl text = "# a\n\n## b\n\n---\n\nabc\nabc\n\nabc\n\nabc\n"
text |> file_system.write_all_text (source_dir </> file_name)
text |> file_system.write_all_text (dist_dir </> file_name)

inl html_path = dist_dir </> $'$"{!file_name}.html"' |> file_system.absolute_path
inl pdf_path = dist_dir </> $'$"{!file_name}.pdf"' |> file_system.absolute_path
inl epub_path = dist_dir </> $'$"{!file_name}.epub"' |> file_system.absolute_path
inl hangul_path = dist_dir </> $'$"{!file_name_no_ext}.hangul.md"' |> file_system.absolute_path
inl hangul_html_path = dist_dir </> $'$"{!file_name_no_ext}.hangul.md.html"' |> file_system.absolute_path
inl hangul_pdf_path = dist_dir </> $'$"{!file_name_no_ext}.hangul.md.pdf"' |> file_system.absolute_path
inl hangul_epub_path = dist_dir </> $'$"{!file_name_no_ext}.hangul.md.epub"' |> file_system.absolute_path

run {
    source_dir
    dist_dir
    cache_dir
    hangul_spec = "por-br"
}
|> async.block_on
|> resultm.unwrap'
|> sm'.format_debug'
|> sm'.from_std_string
|> _assert_eq (
    ;[
        Ok (
            (dist_dir </> file_name |> file_system.absolute_path),
            ;[
                hangul_path
                html_path
                pdf_path
                epub_path
                hangul_html_path
                hangul_pdf_path
                hangul_epub_path
            ]
            |> am'.to_vec |> am'.vec_map (Ok >> resultm.box >> Some >> optionm'.box)
            |> fun x => x : _ (_ (_ _ sm'.std_string))
        )
    ]
    |> am'.to_vec
    |> am'.vec_map resultm.box
    |> fun x => x : _ (_ _ sm'.std_string)
    |> sm'.format_debug'
    |> sm'.from_std_string
)

dist_dir |> file_system.directory_delete true
dist_dir |> file_system.create_directory' |> ignore

text |> file_system.write_all_text (dist_dir </> file_name)

run {
    source_dir = source_dir
    dist_dir = dist_dir
    cache_dir = cache_dir
    hangul_spec = "por-br"
}
|> async.block_on
|> resultm.unwrap'
|> sm'.format_debug'
|> sm'.from_std_string
|> _assert_eq (
    ;[
        Ok (
            (dist_dir </> file_name |> file_system.absolute_path),
            ;[
            ]
            |> am'.to_vec |> am'.vec_map ((optionm.map resultm.box) >> optionm'.box)
        )
    ]
    |> am'.to_vec
    |> am'.vec_map resultm.box
    |> fun x =>
        x : am'.vec (
            resultm.result'
                (string * am'.vec (optionm'.option' (resultm.result' string string)))
                sm'.std_string
        )
    |> sm'.format_debug'
    |> sm'.from_std_string
)

hangul_path
|> file_system.read_all_text
|> _assert_eq "# 아\n\n## 브\n\n---\n\n아브크\n아브크\n\n아브크\n\n아브크\n"

html_path
|> file_system.read_all_text
|> sm'.contains "<p class = \"rule\">***</p>\n<p id = \"para-1\">abc abc</p>"
|> _assert_eq true

pdf_path
|> file_system.read_all_bytes
|> sm'.slice_contains "%PDF-1.5"
|> _assert_eq true

epub_path
|> file_system.read_all_bytes
|> sm'.slice_contains "application/epub+zip"
|> _assert_eq true

hangul_html_path
|> file_system.read_all_text
|> sm'.contains "<p class = \"rule\">***</p>\n<p id = \"para-1\">아브크 아브크</p>"
|> _assert_eq true

hangul_pdf_path
|> file_system.read_all_bytes
|> sm'.slice_contains "%PDF-1.5"
|> _assert_eq true

hangul_epub_path
|> file_system.read_all_bytes
|> sm'.slice_contains "application/epub+zip"
|> _assert_eq true

02:44:48 verbose #1 file_system.create_directory / dir: C:\home\git\polyglot\../vault/target/documents
02:44:48 verbose #2 file_system.create_directory / dir: C:\home\git\polyglot\../vault/target/documents\dist
02:44:48 verbose #3 file_system.create_directory / dir: C:\home\git\polyglot\../vault/target/documents\cache
02:44:49   debug #4 documents.run / source_dir: \\?\C:\home\git\vault\target\documents / dist_dir: \\?\C:\home\git\vault\target\documents\dist / cache_dir: \\?\C:\home\git\vault\target\documents\cache / hangul_spec: "por-br"
02:44:49   debug #5 documents.run / files_len: 1
02:44:49   debug #6 execute_with_options / file_name: git / arguments: ["ls-tree", "--format='%(objectname)'", "origin/gh-pages", "c:/home/git/vault/target/documents/test.md"] / options: (None, "git ls-tree --format='%(objectname)' origin/gh-pages "c:/home/git/vault/target/documents/test.md"", Array(MutCell([])), None, None, Some("\\?\C:\home\git\vault\target\documents"))
02:44:49 verbose #7 execute_wit

## tests

In [ ]:
inl tests () =
    rust.run_tests [
        "verify_app", fun _ =>
            get_command () |> runtime.command_debug_assert
    ]

()



## main

In [ ]:
///! 

inl main (args : array_base string) =
    inl trace_state = get_trace_state ()
    inl trace_level = trace_state.level
    trace_level <- Info

    trace Info
        fun () => $'$"documents.main"'
        fun () => $'$"args: {!args} / {!_locals ()}"'

    inl command = get_command ()
    inl arg_matches = command |> runtime.command_get_matches

    inl source_dir =
        arg_matches
        |> runtime.matches_get_one (get_args () .source_dir)
        |> optionm'.unbox
        |> optionm.value
        |> sm'.from_std_string

    inl dist_dir =
        arg_matches
        |> runtime.matches_get_one (get_args () .dist_dir)
        |> optionm'.unbox
        |> optionm.value
        |> sm'.from_std_string

    inl cache_dir =
        arg_matches
        |> runtime.matches_get_one (get_args () .cache_dir)
        |> optionm'.unbox
        |> optionm.value
        |> sm'.from_std_string

    inl hangul_spec =
        arg_matches
        |> runtime.matches_get_one (get_args () .hangul_spec)
        |> optionm'.unbox
        |> optionm.value
        |> sm'.from_std_string
    
    run { source_dir dist_dir cache_dir hangul_spec }
    |> async.block_on
    |> resultm.unwrap'
    |> ignore

    0i32

inl main () =
    types ()
    $"let tests () = !tests ()" : ()
    $"let main args = !main args" : ()

.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $